**ETF TAX Calculator AUSTRIA**

Data provided by https://my.oekb.at/kapitalmarkt-services/kms-output/fonds-info/sd/af/f?isin=IE00BFY0GT14

Calculation is made in the following way :
$$
\begin{align*}
\text{Capital\_Gain} &= \text{Österreichische\_KESt} \times \text{\#Shares} \\
\text{Taxable\_ETF\_Gain} &= \text{Value\_Year\_After} - \text{Value\_Year\_Before}  \\
\text{Tax\_Paid} &=  \frac {\text{Capital\_Gain}}{\text{USDEUR}} \\
\text{Percentage\_Tax\_Paid} &= \frac {\text{Österreichische KESt} \times 100}{\text{Taxable\_ETF\_Gain} \times \text{USDEUR}} \\
\text{ETF\_New\_Average\_Cost} &= \text{ETF\_Initial\_Cost} + ({\text{Fondsergebnis der Meldeperiode}} \times {\text{USDEUR}})
\end{align*}
$$

Where:


* {Österreichische KESt} is the value provided by the Fund to Öekb (USD).
    * Ertragsteuerliche Behandlung (12. Österreichische KESt, die durch Steuerabzug erhoben wird)
* Value_Year_After  (USD) is the ETF value at the date when KESt is provided.
* Value_Year_Before (USD) is the ETF value one year before.
* Fondsergebnis der Meldeperiode (USD) is the value to add to the original ETF Cost (when you bought the shares)
    * Ertragsteuerliche Behandlung (1. Fondsergebnis der Meldeperiode) 
* USDEUR is computed at the date when KESt is provided.
    * https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html

In [5]:
from modules import general_utils
from modules import stock_utils
import pandas as pd
from pandas import DataFrame
import warnings
import time
import os

**WAY TO AUTOMATIZE**


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains # Importa ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

def get_text_from_element(selectors, max_retries=3):
    service = Service(executable_path=chromedriver_path)
    options = webdriver.ChromeOptions()
    options.binary_location = chrome_path
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)

    retries = 0
    while retries < max_retries:
        try:
            for selector in selectors:
                try:
                    # Use explicit waits to handle dynamic content
                    element = WebDriverWait(driver, 20).until(
                        EC.visibility_of_element_located((By.CSS_SELECTOR, selector))
                    )
                    text = element.text.strip()
                    print(f"Extracted Text using selector '{selector}': {text}")
                    return text
                except (TimeoutException, StaleElementReferenceException):
                    print(f"Selector '{selector}' failed. Retrying...")
                    continue

            raise Exception("All selectors failed to locate the element.")

        except StaleElementReferenceException:
            print(f"Stale element detected. Retry {retries + 1}/{max_retries}...")
            retries += 1
        finally:
            driver.quit()
    

def insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin):
    """Inserisce ISIN, gestisce i cookie, clicca il pulsante e restituisce i risultati della tabella."""

    try:
        
        service = Service(executable_path=chromedriver_path)

        options = Options()
        options.binary_location = chrome_path
        # options.add_argument('--headless=new')
        
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # Gestione del banner dei cookie (con XPath corretto e scroll)
        try:
            # Attesa esplicita per il banner
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@id='cookie-modal']/div/div")))

            cookie_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='cookie-modal']/div/div/div[1]/div[4]/div[2]/p-button/button/span")))
            actions = ActionChains(driver)
            actions.move_to_element(cookie_button).perform()
            cookie_button.click()

            WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.XPATH, "//*[@id='cookie-modal']/div/div"))) #attesa che sparisca il div
        except (TimeoutException, NoSuchElementException) as e:
            print(f"Errore gestione Cookie: {e}")
            driver.save_screenshot("cookie_error.png")
    
        # Inserisci l'ISIN
        isin_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='customer-portal']/div[1]/div/div/app-kms-output-root/lib-kupl-kms-fond-info/lib-bezahl-data/kupl-page-template-sidebar/div/sidebar/div/div[2]/div[2]/div/kupl-autocomplete-filter/div/div/p-autocomplete/div/input"))
        )
        isin_input.clear()
        
        isin_input.send_keys(isin)
        
        isin_to_click = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='pn_id_12_0']/div/b"))
            )
        isin_to_click.click()

        # CSS_SELECTOR is reliable when XPATH has Dynamic ID inside
        
        button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "tr.master span"))
            )
        button.click()
        # Define the selectors (ordered by specificity)
        selectors = [
            # Primary selector: Target by class and parent structure
            "td.highlight.text-right.ng-star-inserted div.ng-star-inserted",
            
            # Fallback 1: Use style attribute for uniqueness
            "td.text-right.ng-star-inserted[style*='width: 125px'] div.ng-star-inserted",
            
            # Fallback 2: Generic numeric value match
            "div.ng-star-inserted:not(:empty):contains(',')"
        ]
        
        try:
            text = get_text_from_element(selectors)
            print(f"Final Value: {text.replace(',', '.')}")  # Optional: Convert comma to decimal
        except Exception as e:
            print(f"Error: {e}")
        
    except TimeoutException as e:    
        print(f"Timeout: {e}")
        #if 'driver' in locals():
            #driver.save_screenshot("timeout.png")
    except Exception as e:
        print(f"Errore generico: {e}")
        #if 'driver' in locals():
            #driver.save_screenshot("errore_generico.png")
    finally:
        if 'driver' in locals():
            driver.quit()


chrome_path = r"C:\Users\david\Downloads\chrome-win64\chrome-win64\chrome.exe"
chromedriver_path = r"C:\Users\david\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
isin_to_insert = "IE00BFY0GT14"
#url = "https://www.pfennigfuchser.at/der-etf-rendite-rechner/"
url = "https://my.oekb.at/kapitalmarkt-services/kms-output/fonds-info/sd/af/f"
insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin_to_insert)


Errore generico: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.264)
Stacktrace:
	GetHandleVerifier [0x00007FF690C280D5+2992373]
	(No symbol) [0x00007FF6908BBFD0]
	(No symbol) [0x00007FF69075590A]
	(No symbol) [0x00007FF69072F4F5]
	(No symbol) [0x00007FF6907D63A7]
	(No symbol) [0x00007FF6907EEE72]
	(No symbol) [0x00007FF6907CF113]
	(No symbol) [0x00007FF69079A918]
	(No symbol) [0x00007FF69079BA81]
	GetHandleVerifier [0x00007FF690C86A2D+3379789]
	GetHandleVerifier [0x00007FF690C9C32D+3468109]
	GetHandleVerifier [0x00007FF690C90043+3418211]
	GetHandleVerifier [0x00007FF690A1C78B+847787]
	(No symbol) [0x00007FF6908C757F]
	(No symbol) [0x00007FF6908C2FC4]
	(No symbol) [0x00007FF6908C315D]
	(No symbol) [0x00007FF6908B2979]
	BaseThreadInitThunk [0x00007FFA0395E8D7+23]
	RtlUserThreadStart [0x00007FFA0599C5DC+44]



In [ ]:
def insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin):
    """Inserisce ISIN, gestisce i cookie, clicca il pulsante e restituisce i risultati della tabella."""
    from selenium.webdriver.common.keys import Keys
    import time

    try:
        service = Service(executable_path=chromedriver_path)
        options = Options()
        options.binary_location = chrome_path
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # Gestione del banner dei cookie
        try:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "cookie-modal")))
            cookie_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, "//button[.//span[contains(text(),'Alle akzeptieren') or contains(text(),'Accept all')]]"))
            )
            cookie_button.click()
            WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "cookie-modal")))
        except Exception as e:
            print(f"Cookie modal non trovato o già gestito: {e}")

        # Inserisci l'ISIN simulando la digitazione lenta
        isin_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder*='ISIN']"))
        )
        isin_input.clear()
        for char in isin:
            isin_input.send_keys(char)
            time.sleep(0.1)  # Simula digitazione umana

        # Attendi che compaia il suggerimento e selezionalo
        suggestion = WebDriverWait(driver, 15).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "div.p-autocomplete-items div"))
        )
        suggestion.click()

        # Attendi che la tabella dei risultati sia caricata
        WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "tr.master span"))
        ).click()

        # Attendi la tabella dettagliata
        table = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "table"))
        )
        print("Tabella trovata!")
        print(table.get_attribute('outerHTML'))  # Per debug

    except Exception as e:
        print(f"Errore: {e}")
        driver.save_screenshot("debug_error.png")
    finally:
        if 'driver' in locals():
            driver.quit()

In [ ]:
def insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin):
    """Inserisce ISIN, gestisce i cookie, clicca il pulsante e restituisce i risultati della tabella."""
    from selenium.webdriver.common.keys import Keys
    import time

    try:
        service = Service(executable_path=chromedriver_path)
        options = Options()
        options.binary_location = chrome_path
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # Gestione del banner dei cookie
        try:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "cookie-modal")))
            cookie_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, "//button[.//span[contains(text(),'Alle akzeptieren') or contains(text(),'Accept all')]]"))
            )
            cookie_button.click()
            WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "cookie-modal")))
        except Exception as e:
            print(f"Cookie modal non trovato o già gestito: {e}")

        # Inserisci l'ISIN simulando la digitazione lenta
        isin_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder*='ISIN']"))
        )
        isin_input.clear()
        for char in isin:
            isin_input.send_keys(char)
            time.sleep(0.1)  # Simula digitazione umana

        # Attendi che compaia il suggerimento e selezionalo
        suggestion = WebDriverWait(driver, 15).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "div.p-autocomplete-items div"))
        )
        suggestion.click()

        # Attendi che la tabella dei risultati sia caricata
        WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "tr.master span"))
        ).click()

        # Attendi la tabella dettagliata
        table = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "table"))
        )
        print("Tabella trovata!")
        print(table.get_attribute('outerHTML'))  # Per debug

    except Exception as e:
        print(f"Errore: {e}")
        driver.save_screenshot("debug_error.png")
    finally:
        if 'driver' in locals():
            driver.quit()

In [ ]:
def insert_isin_and_get_results(url, chrome_path, chromedriver_path, isin):
    """Inserisce ISIN, gestisce i cookie, clicca il pulsante e restituisce i risultati della tabella."""
    from selenium.webdriver.common.keys import Keys
    import time

    try:
        service = Service(executable_path=chromedriver_path)
        options = Options()
        options.binary_location = chrome_path
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # Gestione del banner dei cookie
        try:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "cookie-modal")))
            cookie_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, "//button[.//span[contains(text(),'Alle akzeptieren') or contains(text(),'Accept all')]]"))
            )
            cookie_button.click()
            WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "cookie-modal")))
        except Exception as e:
            print(f"Cookie modal non trovato o già gestito: {e}")

        # Inserisci l'ISIN simulando la digitazione lenta
        isin_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder*='ISIN']"))
        )
        isin_input.clear()
        for char in isin:
            isin_input.send_keys(char)
            time.sleep(0.1)  # Simula digitazione umana

        # Attendi che compaia il suggerimento e selezionalo
        suggestion = WebDriverWait(driver, 15).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "div.p-autocomplete-items div"))
        )
        suggestion.click()

        # Attendi che la tabella dei risultati sia caricata
        WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "tr.master span"))
        ).click()

        # Attendi la tabella dettagliata
        table = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "table"))
        )
        print("Tabella trovata!")
        print(table.get_attribute('outerHTML'))  # Per debug

    except Exception as e:
        print(f"Errore: {e}")
        driver.save_screenshot("debug_error.png")
    finally:
        if 'driver' in locals():
            driver.quit()